In [ ]:
import pandas as pd
import numpy as np
from torch_geometric.data import Data
from sklearn.preprocessing import StandardScaler
import random
from collections import deque
import tensorflow as tf
from tensorflow.keras import layers # type: ignore

class DataPreprocessor:
    def __init__(self, dataset_path, m=3):
        self.dataset_path = dataset_path
        self.m = m
        self.scaler = StandardScaler()

    def load_and_preprocess_data(self):
        df = pd.read_csv(self.dataset_path, usecols=['_source_source_ip', '_source_destination_ip', 
                                                      '_source_network_bytes', '_source_@timestamp', 'label'])
        df['timestamp'] = pd.to_datetime(df['_source_@timestamp'])
        df['window_id'] = df['timestamp'].dt.floor('T').astype('int64') // 10**9 // 60
        grouped = df.groupby('window_id')
        window_ids = np.array(sorted(grouped.groups.keys()))

        X, y = [], []
        for wid in window_ids:
            window_df = grouped.get_group(wid)
            all_ips = np.unique(np.concatenate([window_df['_source_source_ip'].unique(), window_df['_source_destination_ip'].unique()]))
            ip_to_idx = {ip: i for i, ip in enumerate(all_ips)}
            num_nodes = len(all_ips)
            node_features = np.zeros((num_nodes, 4))
            for ip in all_ips:
                src_bytes = window_df[window_df['_source_source_ip'] == ip]['_source_network_bytes'].sum()
                dst_bytes = window_df[window_df['_source_destination_ip'] == ip]['_source_network_bytes'].sum()
                num_src_connections = len(window_df[window_df['_source_source_ip'] == ip])
                num_dst_connections = len(window_df[window_df['_source_destination_ip'] == ip])
                node_features[ip_to_idx[ip]] = [src_bytes, dst_bytes, num_src_connections, num_dst_connections]
            node_features = self.scaler.fit_transform(node_features)
            edge_list = [[ip_to_idx[row['_source_source_ip']], ip_to_idx[row['_source_destination_ip']]] for _, row in window_df.iterrows()]
            edge_index = np.array(edge_list).T
            graph = Data(x=node_features, edge_index=edge_index)
            label = int((window_df['label'] == 'malicious').any())
            X.append(graph)
            y.append(label)

        X_seq, y_seq = [], []
        for k in range(self.m, len(window_ids)):
            seq = X[k - self.m:k]
            sequence_label = int(any(y[k - self.m:k]))
            X_seq.append(seq)
            y_seq.append(sequence_label)

        n = len(X_seq)
        train_end = int(0.8 * n)
        val_end = int(0.9 * n)
        return (X_seq[:train_end], y_seq[:train_end], 
                X_seq[train_end:val_end], y_seq[train_end:val_end], 
                X_seq[val_end:], y_seq[val_end:])


class GCNLayer(layers.Layer):
    def __init__(self, units, activation='relu'):
        super(GCNLayer, self).__init__()
        self.dense = layers.Dense(units, activation=None, use_bias=False,
                                 kernel_regularizer=tf.keras.regularizers.l2(1e-3))
        self.activation = tf.keras.activations.get(activation)
        self.batch_norm = layers.BatchNormalization()

    def call(self, node_features, adj_norm, training=False):
        h = tf.sparse.sparse_dense_matmul(adj_norm, node_features)
        h = self.dense(h)
        h = self.batch_norm(h, training=training)
        h = self.activation(h)
        return h

class GNN(tf.keras.Model):
    def __init__(self, hidden_units=128, output_units=64):
        super(GNN, self).__init__()
        self.gcn1 = GCNLayer(hidden_units)
        self.dropout1 = layers.Dropout(0.3)
        self.gcn2 = GCNLayer(output_units)
        self.dropout2 = layers.Dropout(0.3)

    def call(self, inputs, training=False):
        node_features, edge_indices, num_nodes = inputs
        adj_norm = compute_normalized_adjacency(edge_indices, num_nodes)
        x = self.gcn1(node_features, adj_norm, training=training)
        x = self.dropout1(x, training=training)
        x = self.gcn2(x, adj_norm, training=training)
        x = self.dropout2(x, training=training)
        embedding = tf.reduce_mean(x, axis=0)
        return embedding

def compute_normalized_adjacency(edge_indices, num_nodes):
    num_nodes = tf.cast(num_nodes, tf.int64)
    adj = tf.sparse.SparseTensor(indices=tf.transpose(edge_indices),
                                 values=tf.ones([tf.shape(edge_indices)[1]], dtype=tf.float32),
                                 dense_shape=[num_nodes, num_nodes])
    identity_indices = tf.stack([tf.range(num_nodes), tf.range(num_nodes)], axis=1)
    identity_sparse = tf.sparse.SparseTensor(indices=identity_indices,
                                            values=tf.ones([num_nodes], dtype=tf.float32),
                                            dense_shape=[num_nodes, num_nodes])
    adj = tf.sparse.add(adj, identity_sparse)
    degree = tf.sparse.reduce_sum(adj, axis=1)
    degree_inv_sqrt = tf.pow(degree + 1e-9, -0.5)
    degree_inv_sqrt = tf.where(tf.math.is_inf(degree_inv_sqrt), 0.0, degree_inv_sqrt)
    adj_norm = tf.sparse.SparseTensor(indices=adj.indices,
                                      values=adj.values * tf.gather(degree_inv_sqrt, adj.indices[:, 0]) * tf.gather(degree_inv_sqrt, adj.indices[:, 1]),
                                      dense_shape=adj.dense_shape)
    return adj_norm

class LSTMModel(tf.keras.Model):
    def __init__(self, input_size=64, hidden_size=128):
        super(LSTMModel, self).__init__()
        self.lstm = layers.LSTM(hidden_size, return_sequences=False,
                               kernel_regularizer=tf.keras.regularizers.l2(1e-3),
                               recurrent_regularizer=tf.keras.regularizers.l2(1e-3))
        self.dense = layers.Dense(1, activation='sigmoid',
                                 kernel_regularizer=tf.keras.regularizers.l2(1e-3))
        self.batch_norm = layers.BatchNormalization()

    def call(self, inputs, training=False):
        x = self.lstm(inputs)
        x = self.batch_norm(x, training=training)
        x = self.dense(x)
        return x

class RLTrainer:
    def __init__(self, X_train, y_train, X_val, y_val, model_path, m=3, num_episodes=100, batch_size=32, 
                 hidden_units=128, output_units=64, gamma=0.99, epsilon=1.0, epsilon_min=0.01, epsilon_decay=0.995):
        self.X_train, self.y_train = X_train, y_train
        self.X_val, self.y_val = X_val, y_val
        self.model_path = model_path
        self.m = m
        self.num_episodes = num_episodes
        self.batch_size = batch_size
        self.gnn = GNN(hidden_units=hidden_units, output_units=output_units)
        self.lstm = LSTMModel(input_size=output_units, hidden_size=128)
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.00005)
        self.best_val_loss = float('inf')
        self.counter = 0
        self.patience = 10
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.memory = deque(maxlen=2000)
        self.target_gnn = GNN(hidden_units=hidden_units, output_units=output_units)
        self.target_lstm = LSTMModel(input_size=output_units, hidden_size=128)
        self.update_target_model()

    def update_target_model(self):
        self.target_gnn.set_weights(self.gnn.get_weights())
        self.target_lstm.set_weights(self.lstm.get_weights())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(2)
        embeddings = self.get_embeddings(state)
        q_values = self.lstm(embeddings)
        return 0 if q_values.numpy()[0][0] < 0.5 else 1

    def get_embeddings(self, sequence):
        embeddings = []
        for graph in sequence:
            node_features = tf.convert_to_tensor(graph['x'], dtype=tf.float32)
            edge_indices = tf.convert_to_tensor(graph['edge_index'], dtype=tf.int64)
            num_nodes = node_features.shape[0]
            embedding = self.gnn((node_features, edge_indices, num_nodes))
            embeddings.append(embedding)
        return tf.stack(embeddings)[None, :]

    def replay(self):
        if len(self.memory) < self.batch_size:
            return
        minibatch = random.sample(self.memory, self.batch_size)
        states = [m[0] for m in minibatch]
        actions = [m[1] for m in minibatch]
        rewards = [m[2] for m in minibatch]
        next_states = [m[3] for m in minibatch]
        dones = [m[4] for m in minibatch]

        state_embeddings = tf.stack([self.get_embeddings(s)[0] for s in states])
        next_state_embeddings = tf.stack([self.get_embeddings(ns)[0] for ns in next_states])
        
        with tf.GradientTape() as tape:
            q_values = self.lstm(state_embeddings)
            target_q_values = self.target_lstm(next_state_embeddings)
            targets = []
            for i in range(self.batch_size):
                target = rewards[i] if dones[i] else rewards[i] + self.gamma * tf.reduce_max(target_q_values[i])
                targets.append(target)
            targets = tf.convert_to_tensor(targets, dtype=tf.float32)[:, None]
            loss = tf.keras.losses.mean_squared_error(targets, q_values)
        
        grads = tape.gradient(loss, self.gnn.trainable_variables + self.lstm.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.gnn.trainable_variables + self.lstm.trainable_variables))
        
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def train(self):
        for episode in range(self.num_episodes):
            train_data = list(zip(self.X_train, self.y_train))
            random.shuffle(train_data)
            total_reward = 0
            for i, (sequence, label) in enumerate(train_data):
                action = self.act(sequence)
                reward = 1 if action == label else -1
                total_reward += reward
                next_idx = min(i + 1, len(train_data) - 1)
                next_sequence, _ = train_data[next_idx]
                done = i == len(train_data) - 1
                self.remember(sequence, action, reward, next_sequence, done)
                self.replay()

            val_loss = 0
            for sequence, label in zip(self.X_val, self.y_val):
                embeddings = self.get_embeddings(sequence)
                q_values = self.lstm(embeddings)
                pred = 0 if q_values.numpy()[0][0] < 0.5 else 1
                val_loss += tf.keras.losses.binary_crossentropy([label], [pred]).numpy()
            val_loss /= len(self.X_val)
            print(f"Episode {episode}, Reward: {total_reward}, Val Loss: {val_loss:.4f}, Epsilon: {self.epsilon:.4f}")

            if val_loss < self.best_val_loss:
                self.best_val_loss = val_loss
                self.counter = 0
                self.gnn.save_weights(f"{self.model_path}_gnn.h5")
                self.lstm.save_weights(f"{self.model_path}_lstm.h5")
                self.update_target_model()
            else:
                self.counter += 1
                if self.counter >= self.patience:
                    print("Early stopping triggered")
                    break

        self.gnn.load_weights(f"{self.model_path}_gnn.h5")
        self.lstm.load_weights(f"{self.model_path}_lstm.h5")

class ModelEvaluator:
    def __init__(self, gnn, lstm):
        self.gnn = gnn
        self.lstm = lstm

    def evaluate(self, X_val, y_val):
        val_loss = 0
        correct = 0
        total = 0
        for sequence, label in zip(X_val, y_val):
            embeddings = []
            for graph in sequence:
                node_features = tf.convert_to_tensor(graph['x'], dtype=tf.float32)
                edge_indices = tf.convert_to_tensor(graph['edge_index'], dtype=tf.int64)
                num_nodes = node_features.shape[0]
                embedding = self.gnn((node_features, edge_indices, num_nodes))
                embeddings.append(embedding)
            sequence_embeddings = tf.stack(embeddings)[None, :]
            y_pred = self.lstm(sequence_embeddings).numpy()[0][0]
            val_loss += tf.keras.losses.binary_crossentropy([label], [y_pred]).numpy()
            if (y_pred > 0.5) == label:
                correct += 1
            total += 1
        val_loss /= total
        val_accuracy = correct / total
        return val_loss, val_accuracy

class ModelTester:
    def __init__(self, model):
        self.gnn, self.lstm = model
        self.scaler = StandardScaler()

    def test(self, X_test, y_test):
        correct = 0
        total = 0
        for sequence, label in zip(X_test, y_test):
            embeddings = []
            for graph in sequence:
                node_features = tf.convert_to_tensor(graph['x'], dtype=tf.float32)
                edge_indices = tf.convert_to_tensor(graph['edge_index'], dtype=tf.int64)
                num_nodes = node_features.shape[0]
                embedding = self.gnn((node_features, edge_indices, num_nodes))
                embeddings.append(embedding)
            sequence_embeddings = tf.stack(embeddings)[None, :]
            y_pred = self.lstm(sequence_embeddings).numpy()[0][0]
            if (y_pred > 0.5) == label:
                correct += 1
            total += 1
        test_accuracy = correct / total
        print(f"Test Accuracy: {test_accuracy:.4f}")
        return test_accuracy

if __name__ == '__main__':
    dataset_path = 'C:\\Users\\ASUS\\Guidewire_Hackathon\\datasets\\elastic_may2021_malicious_data.csv'
    test_dataset_path = 'C:\\Users\\ASUS\\Guidewire_Hackathon\\datasets\\elastic_may2022_data.csv'
    model_path = 'C:\\Users\\ASUS\\Guidewire_Hackathon\\src\\models\\trained_hybrid_model1'
    preprocessor = DataPreprocessor(dataset_path, m=3)
    X_train, y_train, X_val, y_val, X_test, y_test = preprocessor.load_and_preprocess_data()
    trainer = RLTrainer(X_train, y_train, X_val, y_val, model_path, m=3, num_episodes=50, batch_size=32, 
                         hidden_units=128, output_units=64)
    trainer.train()
    evaluator = ModelEvaluator(trainer.gnn, trainer.lstm)
    val_loss, val_accuracy = evaluator.evaluate(X_val, y_val)
    tester = ModelTester((trainer.gnn, trainer.lstm))
    test_accuracy = tester.test(X_test, y_test)
    new_test_accuracy = tester.preprocess_and_test(test_dataset_path, m=3)
    print(f"New Test Accuracy: {new_test_accuracy:.4f}")
